# Gerador de lat e log 

....

## importação dos dados

In [1]:
import pandas as pd
from glob import glob
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from numpy import dtype

In [2]:
#importação de dados
arquivos = sorted(glob(r'C:\Users\otavi\Desktop\base\*.csv'))
df_original = pd.concat((pd.read_csv(cont, 
    dtype={'CNES_EXECUTANTE':'str','IBGE_USUARIO':'str','CEP_USUARIO':'str', 'CEP_UNID_EXECUTANTE':'str'}) 
    for cont in arquivos),ignore_index=True)


In [3]:
df_cep = pd.read_csv(r'C:\Users\otavi\GitHub\SISREG_Estudo_caso-AI_FALTAS_USUARIOS\dados\TABELA_CEP_BR.txt', dtype=str, sep="	", header=None)
df_cep.columns = ['CEP','UF','CIDADE','BAIRRO','LOGRADOURO']  #http://cep.la/baixar

## identificando CEP's unicos

### CEP's do USUARIO

In [4]:
#criando um DataFrame somente os CEP's dos usuarios.
df_ceps_usuario = df_original['CEP_USUARIO']
df_ceps_usuario = pd.DataFrame(df_ceps_usuario)
df_ceps_usuario.drop_duplicates(inplace=True)
df_ceps_usuario.dropna(inplace=True)
df_ceps_usuario.head(5)


,CEP_USUARIO
0,12441170
1,12922810
2,66033540
3,54515090
4,88352495


In [5]:
#quantidade de CEP's localizado na base 
print("Cep's unicos de usuarios: ",df_ceps_usuario.count())

Cep's unicos de usuarios:  CEP_USUARIO    413882
dtype: int64


### CEP's da UNIDADE EXECUTANTE

In [6]:
#criando um DataFrame somente os CEP's das unidades.
df_ceps_cnes = df_original['CEP_UNID_EXECUTANTE']
df_ceps_cnes = pd.DataFrame(df_ceps_cnes)
df_ceps_cnes.drop_duplicates(inplace=True)
df_ceps_cnes.dropna(inplace=True)
df_ceps_cnes.head(5)

,CEP_UNID_EXECUTANTE
0,12410040
1,12900005
2,67130130
3,54520250
4,88350170


In [7]:
#quantidade de CEP's localizado na base e tratado
print("Cep's unicos de usuarios: ",df_ceps_cnes.count())

Cep's unicos de usuarios:  CEP_UNID_EXECUTANTE    2415
dtype: int64


## TRATAMENTO COM OS CEP's DOS USUARIOS

In [8]:
#localizando endereço do usuario com a base do CEP's BR
df_ceps_usuario = pd.merge(df_ceps_usuario, df_cep, left_on=['CEP_USUARIO'],right_on=['CEP'] , how='left')

In [9]:
#criando um endereço completo para realizar pesquisa de lat e log
df_ceps_usuario["END_COMPLETO"] = df_ceps_usuario['LOGRADOURO']+ " - " + df_ceps_usuario['CIDADE']+ " - " + df_ceps_usuario['UF']
df_ceps_usuario.dropna(inplace=True)
df_ceps_usuario.head(5)

,CEP_USUARIO,CEP,UF,CIDADE,BAIRRO,LOGRADOURO,END_COMPLETO
0,12441170,12441170,SP,Pindamonhangaba,Jardim Padre Rodolfo,Rua Oswaldo Marcondes de Azevedo,Rua Oswaldo Marcondes de Azevedo - Pindamonhan...
1,12922810,12922810,SP,Bragança Paulista,Parque dos Estados,Rua Amapá,Rua Amapá - Bragança Paulista - SP
2,66033540,66033540,PA,Belém,Condor,Passagem Nazaré,Passagem Nazaré - Belém - PA
3,54515090,54515090,PE,Cabo de Santo Agostinho,COHAB,Rua Alexandra Figueiredo Silva,Rua Alexandra Figueiredo Silva - Cabo de Santo...
4,88352495,88352495,SC,Brusque,Limoeiro,Rua Itajaí,Rua Itajaí - Brusque - SC


In [10]:
#localizando lat e log (PROCESSO DEMORADO)
locator = Nominatim(user_agent='localizacao')
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)
df_ceps_usuario["GEOLOCALIZADO"] = df_ceps_usuario["END_COMPLETO"].apply(geocode)
df_ceps_usuario.head(10)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Conjunto Conjunto I - Brasília - DF',), **{}).
Traceback (most recent call last):
  File "C:\Users\otavi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\otavi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2288.0_x64__qbz5n2kfra8p0\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2288.0_x64__qbz5n2kfra8p0\lib\http\client.py", line 318, in begin
    version, statu

KeyboardInterrupt: 

In [ ]:
#verificar LOCALIZAÇÃO lat e log não localizada (limpeza)
df_ceps_usuario.dropna(inplace=True)

In [ ]:
#colocando a LATITUDE e LONGITUDE na tabela
df_ceps_usuario['LATITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.latitude)
df_ceps_usuario['LONGITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.longitude)
df_ceps_usuario.drop('END_COMPLETO', axis=1, inplace=True)
df_ceps_usuario.drop('GEOLOCALIZADO', axis=1, inplace=True)
df_ceps_usuario.head(10)

In [ ]:
#quantidade de CEP's localizado na base e tratado
print("Cep's unicos de usuarios: ",df_ceps_usuario.count())

## GERAÇÃO DE ARQUIVO CSV - CEP'S com latitude e longitude 

In [ ]:
df_ceps_tratado = df_ceps_usuario.filter(items=['CEP','UF','CIDADE','BAIRRO','LOGRADOURO','LATITUDE_USUARIO','LONGITUDE_USUARIO'])
df_ceps_tratado

In [ ]:
df_ceps_tratado.to_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_CEP_LAT_LON.csv')